In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

/home/ja/projects/sentiment/GUiIO_pt2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-01 21:38:11.524464: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-01 21:38:11.699923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748806691.765707   14551 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748806691.786038   14551 cuda_

In [2]:
#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

In [3]:
outfile_name = 'test_embeddings'

In [2]:
csv_training_data = pd.read_csv("../data/test_data.csv")
csv_training_data.head()

,text,sentiment
0,yes black red fav colors really wanted colors ...,0
1,qu buy open house weekend ampm best value one ...,1
2,fran greet air thats okay hahahaha thanks,1
3,omg thats awesome goood luck u meet caaute cal...,1
4,hahahhaha good point,1


In [5]:
texts = csv_training_data["text"].tolist()
texts_str = [str(x) for x in texts]
texts_str

['yes black red fav colors really wanted colors def looks awesome jared',
 'qu buy open house weekend ampm best value one bedroom lic long island city bd',
 'fran greet air thats okay hahahaha thanks',
 'omg thats awesome goood luck u meet caaute calgary boys buying clothes tell lol',
 'hahahhaha good point',
 'good night twitterers ill see yall tomorrow rise early',
 'na cday hurr mama',
 'whyyyyy guys vegas last week blast',
 'another thought justice cover voyager superb electro bass',
 'un gusto',
 'time go pay ac car fixed wait didnt already yes maybe second time charm',
 'nails came lt im watching enchanted coollt cheered since nails went wick',
 'get followers day using add everyone train pay vip',
 'congrats anxiously await seems far',
 'young man love feeling ancient moment good night',
 'watching quothunt red octoberquot feet wet day outsidenothing better',
 'ahaha sounds like go buy tshirt come home new pair heels',
 'life one big drama love gonna miss girlies lesbian kisses'

In [6]:
embeddings_arr = model.encode(texts_str, convert_to_numpy=True, device='cuda', show_progress_bar=True)


Batches: 100%|██████████| 10000/10000 [03:07<00:00, 53.43it/s]


In [7]:
embeddings_arr.shape

(320000, 768)

In [8]:
with open(f'{outfile_name}.npy', 'wb') as f:
    np.save(f, embeddings_arr)

In [9]:
embedddings_df = csv_training_data.copy(deep=True)
embedddings_df['embeddings'] = embedddings_df.apply(lambda row: embeddings_arr[row.name], axis=1)
embedddings_df.head()

,text,sentiment,embeddings
0,yes black red fav colors really wanted colors ...,0,"[-0.015319691, 0.029470563, -0.010942006, 0.02..."
1,qu buy open house weekend ampm best value one ...,1,"[-0.055580236, 0.019168517, 0.008300844, 0.002..."
2,fran greet air thats okay hahahaha thanks,1,"[0.05550304, 0.033181395, -0.027894102, 0.0079..."
3,omg thats awesome goood luck u meet caaute cal...,1,"[0.0014827816, 0.09667131, -0.046894766, 0.017..."
4,hahahhaha good point,1,"[0.019305877, 0.13698456, -0.04215252, 0.01921..."


In [10]:
embedddings_df.to_feather(f'{outfile_name}.feather', compression='zstd')